In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('../CBlog2/predicting_heart_failure/heart_failure_clinical_records_dataset.csv')

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#Loop to seperate X data into categorical and numerical colu
categoricals_columns = []
numericals_columns = []
for i in list(X.columns):
    if data[i].nunique() > 2:
        numericals_columns.append(i)
    else:
        categoricals_columns.append(i)

categoricals = X[categoricals_columns]
numericals = X[numericals_columns]